<h3 style="color: pink;">PROYECTO 2 ESTADÍSTICA</h3>

<h4> Importación librerias

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import scipy.stats as stats

<h4>Funciones

<h10>En caso de que la columna no tome los valores como int o float los convierte a valores numéricos, luego convierte los valores NaN de una columna a su media.

<h10>Con el ingreso de una serie de datos, devuelve una lista con la media, mediana, moda, SD, MAD, varianza, IQR, CV y CVM.

In [28]:
def informacion(dat,categorico=False):
    lista = []    
    if categorico == False:
        # Intentamos tratar los datos como numéricos
        dat = pd.to_numeric(dat)
        lista.append(np.mean(dat))  # media
        lista.append(np.median(dat))  # mediana
        moda = dat.mode().iloc[0] if not dat.mode().empty else np.nan #Si no es categorico calcula la moda
        lista.append(moda)
        lista.append(np.std(dat))  # SD
        lista.append(stats.median_abs_deviation(dat))  # MAD
        lista.append(np.var(dat))  # Varianza
        Q1 = np.percentile(dat, 25)
        Q3 = np.percentile(dat, 75)
        lista.append(Q3 - Q1)  # IQR
        lista.append(dat.std(ddof=1) / dat.mean() * 100)  # CV
        lista.append((stats.median_abs_deviation(dat)/ np.median(dat)) if np.median(dat) != 0 else np.nan)  # CVM
    elif categorico == True:
        lista = [np.nan] * 9  
        lista[2] = dat.mode().iloc[0] if not dat.mode().empty else np.nan  
    
    return lista


<h10>Con el ingreso de una serie de datos, devuelve una lista con los datos atípicos y la cantidad de estos.  

In [5]:
def atipicos (valor):  
    atipico = []
    valor = pd.to_numeric(valor)
    Q1 = np.nanpercentile(valor,25)    
    Q3 =np.nanpercentile(valor,75)
    iqr = Q3-Q1

    for x in valor:
        if x < (Q1 - 1.5*iqr) :
            atipico.append(x)
        elif x > (Q3 + 1.5*iqr):
            atipico.append(x)

    cont = len(atipico)
    return  cont,atipico

<h10>Recibe una lista de variables y un dataframe. Muestra los datos atípicos y la cantidad para cada variable.

In [6]:
def mostrar_atipicos(lista : list, df):
    for i in range(len(lista)):
        cont,datos = atipicos(df[lista[i]])
        print(f"{lista[i]} tiene {cont} datos atipicos")
        print("Los datos atipicos son:")
        print(datos)

Diccionario para almacenar las variables y sus índices (columnas del CSV).

In [17]:
cols = {'aveOralM':115,
'Gender':116, 
'Age':117, 
'Ethnicity':118,
'T_atm':119, 
'Humidity':120,
'Cosmetics':122, 
'Max1R13_1':3,
'Max1R13_2':31,
'Max1R13_3':59,
'Max1R13_4': 87,
'Max1L13_1':5,
'Max1L13_2':33,
'Max1L13_3':61,
'Max1L13_4':89,
'T_Max_1': 27,
'T_Max_2': 55,
'T_Max_3': 83,
'T_Max_4': 111,
'TF_HCC_1': 20,
'TF_HCC_2': 48,
'TF_HCC_3': 76,
'TF_HCC_4': 104
}

In [8]:
variables_numericas = [
    'aveOralM',  
    'T_atm', 
    'Humidity',
    'PromMax1R13',
    'PromMax1L13',	
    'PromT_Max',
    'PromTF_HCC'
]


In [9]:
variables_categoricas = ['Gender',
'Age',
'Ethnicity',
'Cosmetics']

Lectura de la base de  datos y conversión a DataFrame

In [10]:
csv = pd.read_csv(r'FLIR_groups1and2.csv', delimiter=';')
csv = csv.iloc[2:].reset_index(drop=True)
 

In [25]:
# Lista de columnas a extraer
columnas = ['aveOralM', 'T_atm', 'Humidity', 'Gender', 'Age', 'Ethnicity', 'Cosmetics']
indices = [cols[col] for col in columnas]

csv_filter = csv.iloc[:, indices].copy()
csv_filter.columns = columnas


Para **promediar las 4 zonas**, se crea una nueva función que:
 
Extrae columnas desde un DataFrame según los índices dados,
convierte a numérico, reemplaza NaNs con la media,
y devuelve una Serie con el promedio por fila.
    
    Parámetros:
    - df: DataFrame original.
    - indices_dict: diccionario con los índices de columnas.
    - nombres_cols: lista con las llaves a extraer del diccionario.
    - nuevo_nombre: nombre para la nueva columna de promedio (usado externamente).



In [1]:
def extraer_y_promediar(df, indices_dict, nombres_cols):
    columnas = [indices_dict[n] for n in nombres_cols]
    data_temp = df.iloc[:, columnas].copy()
    data_temp.columns = nombres_cols

    for col in nombres_cols:
        data_temp[col] = pd.to_numeric(data_temp[col], errors='coerce')
        data_temp[col] = data_temp[col].fillna(data_temp[col].mean())

    return data_temp.mean(axis=1)


Agregar las columnas de promedios al dataframe original

In [32]:
csv_filter.loc[:, 'PromMax1R13'] = extraer_y_promediar(csv, cols, ['Max1R13_1','Max1R13_2','Max1R13_3','Max1R13_4'])
csv_filter.loc[:, 'PromMax1L13'] = extraer_y_promediar(csv, cols, ['Max1L13_1','Max1L13_2','Max1L13_3','Max1L13_4'])
csv_filter.loc[:, 'PromT_Max'] = extraer_y_promediar(csv, cols, ['T_Max_1','T_Max_2','T_Max_3','T_Max_4'])
csv_filter.loc[:, 'PromTF_HCC'] = extraer_y_promediar(csv, cols, ['TF_HCC_1','TF_HCC_2','TF_HCC_3','TF_HCC_4'])

Convierte los np.nan de la columna de cosmetics en string 'Nan' para evitar problemas al graficar

In [ ]:
csv_filter["Cosmetics"] = csv_filter["Cosmetics"].fillna('Nan')

Convierte los valores de las columnas numéricas a floats o ints para evitar problemas al graficar

In [24]:
for i in variables_numericas:
    csv_filter[i] = pd.to_numeric(csv_filter[i],errors='coerce')

CSV con los datos de interés

In [23]:
csv_filter.to_csv('FLIR_groups1and2_replace.csv',  sep=';', decimal=',',index=False)

1. **Analisis exploratorio variables nuevas** 

In [34]:
resultados = {'': ["Media", "Mediana", "Moda", "SD", "MAD", "Varianza", "IQR", "CV", "CVM"],    
    'aveOralM': informacion(csv_filter['aveOralM']),
    'Gender': informacion(csv_filter['Gender'], categorico=True),
    'Age': informacion(csv_filter['Age'], categorico=True),
    'Ethnicity': informacion(csv_filter['Ethnicity'], categorico=True), 
    'T_atm': informacion(csv_filter['T_atm']),
    'Humidity': informacion(csv_filter['Humidity']),
    'Cosmetics': informacion(csv_filter['Cosmetics'], categorico=True),
    'PromMax1R13': informacion(csv_filter['PromMax1R13']),
    'PromMax1L13': informacion(csv_filter['PromMax1L13']),	
    'PromT_Max': informacion(csv_filter['PromT_Max']),
    'PromTF_HCC': informacion(csv_filter['PromTF_HCC'])
}

Convertir resultados a Df

In [ ]:
df_estadisticas = pd.DataFrame(resultados)
print("Resumen estadístico completo:\n")
print(df_estadisticas)

Resumen estadístico completo:

              aveOralM  Gender    Age Ethnicity      T_atm    Humidity  \
0     Media  37.028382     NaN    NaN       NaN  24.115392   28.723039   
1   Mediana  36.940000     NaN    NaN       NaN  24.000000   26.300000   
2      Moda  36.890000  Female  18-20     White  24.000000   30.000000   
3        SD   0.509252     NaN    NaN       NaN   1.335682   13.065218   
4       MAD   0.200000     NaN    NaN       NaN   0.600000    8.900000   
5  Varianza   0.259338     NaN    NaN       NaN   1.784047  170.699910   
6       IQR   0.362500     NaN    NaN       NaN   1.300000   18.600000   
7        CV   1.375978     NaN    NaN       NaN   5.541430   45.509205   
8       CVM   0.005414     NaN    NaN       NaN   0.025000    0.338403   

  Cosmetics  PromMax1R13  PromMax1L13  PromT_Max  PromTF_HCC  
0       NaN    35.597916    34.889475  35.809334   34.567550  
1       NaN    35.550000    34.916250  35.798750   34.597500  
2         0    35.677500    35.175000  

**Comparación SD Y MAD**
